In [2]:
#extract features
import numpy as np
def extract_label(filename):
    parts = filename.split("_")
    if parts[1] == "B":
        label = 1
    else:
        label = 0
    sub_parts = parts[2].split("-")
    size = np.int(sub_parts[3])
    return label, size

In [3]:
#prepare files in csv file
import os
src_directory = "/tf/dataset/classes"
dest_directory = "/tf/dataset/classes"
sub_directory = ["40_patch_4","100_patch_4","200_patch_4","400_patch_4"]
for sub_dir in sub_directory:
    curr_sub_dir = os.path.join(src_directory,sub_dir)
    curr_filename = "data_file_" + sub_dir + ".csv"
    #curr_filename = "data_file_Fake_" + sub_dir + ".csv"
    content = []
    # extract image label
    for path,_,ls_files in os.walk(curr_sub_dir):
        for file in ls_files:
            label, size = extract_label(file)
            line = ("{},{},{}".format(os.path.join(curr_sub_dir,file),label,size)) 
            content.append(line)
    # write to file        
    with open(os.path.join(dest_directory,curr_filename), 'w') as writer:
        for line in content:
            writer.write(line + "\n")    



In [1]:
# test generated label data
import sys
sys.path.append('../') 
from codes.data.create_dataset_01 import create_dataset
directory = "/tf/dataset/classes"
dataset = create_dataset(directory)

In [2]:
from sklearn.model_selection import train_test_split
directory = "F:\\GraduateClass\\Thesis\\Dataset\\BreaKHis\\histology_slides\\classes"
data_file_list = ["data_file_40.csv","data_file_100.csv","data_file_200.csv","data_file_400.csv"]
rate = 0.7
epoches = 5
for epoch in range(epoches):
    fn_train_ep = "train_data_file_ep_{}.csv" . format(epoch)
    fn_test_ep = "test_data_file_ep_{}.csv" . format(epoch)
    print("Epoch {}" . format(epoch))
    for data_file in data_file_list:
        with open(os.path.join(directory,data_file), "r") as f:
            lines = f.readlines()
        sub_train, sub_test = train_test_split(lines,test_size=(1-rate))

        with open(os.path.join(directory,fn_train_ep), 'a') as writer1:
            for line in sub_train:
                writer1.write(line)        

        with open(os.path.join(directory,fn_test_ep), 'a') as writer2:
            for line in sub_test:
                writer2.write(line)

tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(100, shape=(), dtype=int32)
tf.Tensor(100, shape=(), dtype=int32)
tf.Tensor(400, shape=(), dtype=int32)
tf.Tensor(200, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(200, shape=(), dtype=int32)
tf.Tensor(200, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(100, shape=(), dtype=int32)
tf.Tensor(200, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(400, shape=(), dtype=int32)
tf.Tensor(200, shape=(), dtype=int32)
tf.Tensor(100, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(400, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(100, shape=(), dtype=int32)
tf.Tensor(200, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(100, shape=(),

In [4]:
# divide images into equal-size images
import os
from PIL import Image
import numpy as np
src_path = "/tf/dataset/classes/400"
dest_path = "/tf/dataset/classes/400_patch_9"
photo_size = (460,700)
division = 2
size_rows = np.int(photo_size[0]/division)
size_cols = np.int(photo_size[1]/division)

for path,_,files_list in os.walk(src_path):
    
    for filename in files_list:
        
        filename_no_ext = filename.split(".")[0]
        
        in_im = Image.open(os.path.join(src_path,filename))
        
        imarray = np.array(in_im)
        stt = 0
        for i in range(division):
            for j in range(division):
                sub_imarray = imarray[i*size_rows:i*size_rows+size_rows,j*size_cols:j*size_cols+size_cols,:]
                ou_im = Image.fromarray(sub_imarray)
                ou_im.save(os.path.join(dest_path,"{}-{}.png" 
                                        . format(filename_no_ext,stt)))
                stt += 1

In [8]:

directory = "/tf/dataset/classes"
epoch = 2
from sklearn.model_selection import train_test_split

full_img_data = ['40','100','200','400']

for i_ep in range(epoch):
    train_filename = "train_data_file_ep_{}.csv" . format(i_ep)
    test_filename = "test_data_file_ep_{}.csv" . format(i_ep)
    f_train = open(os.path.join(directory,'epoch',train_filename),'a')
    f_test = open(os.path.join(directory,'epoch',test_filename),'a')
    for img_size in full_img_data:

        #read full images
        list_img = []
        with open(os.path.join(directory,'data_file_{}.csv' . format(img_size)), "r") as f:
            lines = f.readlines()
        for ln in lines:
            line = ln.strip().split('/')[-1]
            list_img.append(line)

        #convert images into 4 patch images
        d_train, d_test = train_test_split(list_img,test_size=0.3, random_state=42)
        for x in d_train:
            label, size = extract_label(x)
            fn_without_ext = x.split(".")[0]
            for i_patch in range(4):
                fn_with_ext = "{}/{}-{}.png,{},{}" .format(os.path.join(directory,img_size+'_patch') ,fn_without_ext,i_patch,label,size)
                f_train.write(fn_with_ext + "\n")

        for y in d_test:
            fn_without_ext = y.split(".")[0]
            label, size = extract_label(y)
            for i_patch in range(4):
                fn_with_ext = "{}/{}-{}.png,{},{}" .format(os.path.join(directory,img_size+'_patch') ,fn_without_ext,i_patch,label,size)
                f_test.write(fn_with_ext + "\n")
            
     

In [5]:
# divide images into 2 images per horizontally, vertically
import os
from PIL import Image
import numpy as np
src_path = "/tf/dataset/classes/400"
dest_path = "/tf/dataset/classes/400_patch_4"
photo_size = (460,700)
division = 2
size_rows = np.int(photo_size[0]/division)
size_cols = np.int(photo_size[1]/division)

for path,_,files_list in os.walk(src_path):
    
    for filename in files_list:
        
        filename_no_ext = filename.split(".")[0]
        
        in_im = Image.open(os.path.join(src_path,filename))
        
        imarray = np.array(in_im)
        stt = 0
        #divide image by horizontally and then vertically axis
        for i in range(division):
            sub_imarray_horizontal = imarray[i*size_rows:i*size_rows+size_rows,:,:]
            sub_imarray_vertical = imarray[:,i*size_cols:i*size_cols+size_cols,:]

            ou_im_horizontal = Image.fromarray(sub_imarray_horizontal)
            ou_im_horizontal.save(os.path.join(dest_path,"{}-{}.png" 
                                    . format(filename_no_ext,stt)))
            ou_im_vertical = Image.fromarray(sub_imarray_vertical)
            ou_im_vertical.save(os.path.join(dest_path,"{}-{}.png" 
                                    . format(filename_no_ext,stt+2)))
            stt += 1

In [14]:
#create fake file csv from all fake images generated by stylegan

#extract features
import numpy as np
def extract_fake_label(filename):
    parts = filename.split("_")
    if parts[1] == "B":
        label = 1
    else:
        label = 0
    sub_parts = parts[2].split("-")
    size = np.int(sub_parts[3])
    return label, size

import os
dest_path = "/tf/dataset/classes/"
content = []
output_filename = 'data_file_fake.csv'

#Write B cancer
src_path = "/tf/Downloads/b_output_stylegan/"
folders = ['B100','B40','B200','B400']
for dir_path in folders:
    for path,_,files_list in os.walk(os.path.join(src_path,dir_path)):
        for filename in files_list:
            
            label, size = extract_fake_label(filename)
            full_filename = os.path.join(path,filename)
            line = ("{},{},{}".format(full_filename,label,size)) 
            content.append(line)
#Write M cancer
src_path = "/tf/Downloads/m_output_stylegan/"
folders = ['M100','M40','M200','M400']
for dir_path in folders:
    for path,_,files_list in os.walk(os.path.join(src_path,dir_path)):
        for filename in files_list:
            
            label, size = extract_fake_label(filename)
            
            full_filename = os.path.join(path,filename)
            line = ("{},{},{}".format(full_filename,label,size)) 
            content.append(line)            
                        
# write all to file        
with open(os.path.join(dest_path,output_filename), 'w') as writer:
    for line in content:
        writer.write(line + "\n")                

In [10]:
filename = 'fake_B_F-F-F-400-433.png'

parts = filename.split("_")
if parts[1] == "B":
    label = 1
else:
    label = 0
sub_parts = parts[2].split("-")
size = np.int(sub_parts[3])
print(label,size)

1 400


In [4]:
#divide BACH images

# divide images into 2 images per horizontally, vertically
import os
from PIL import Image
import numpy as np
src_path = "/tf/dataset/BACH/Invasive"
dest_path = "/tf/dataset/BACH/Invasive_png"
photo_size = (2048,1536)
division = 2
size_rows = np.int(photo_size[0]/division)
size_cols = np.int(photo_size[1]/division)

for path,_,files_list in os.walk(src_path):
    
    for filename in files_list:
        if filename.endswith('.tif'):
            filename_no_ext = filename.split(".")[0]
            #print(filename_no_ext)
            in_im = Image.open(os.path.join(src_path,filename))

            imarray = np.array(in_im)
            stt = 0
            #divide image by horizontally and then vertically axis
            for i in range(division):
                sub_imarray_horizontal = imarray[i*size_rows:i*size_rows+size_rows,:,:]
                sub_imarray_vertical = imarray[:,i*size_cols:i*size_cols+size_cols,:]

                ou_im_horizontal = Image.fromarray(sub_imarray_horizontal)
                ou_im_horizontal.save(os.path.join(dest_path,"{}-{}.png" 
                                        . format(filename_no_ext,stt)))
                ou_im_vertical = Image.fromarray(sub_imarray_vertical)
                ou_im_vertical.save(os.path.join(dest_path,"{}-{}.png" 
                                        . format(filename_no_ext,stt+2)))
                stt += 1

In [7]:
#create BACH file csv for training

#extract features
import numpy as np
def extract_BACH_label(filename):
    parts = filename
    if parts[0] == "b" or parts[0] == "n":
        label = 1
    else:
        label = 0
    return label

import os
src_directory = "/tf/dataset/BACH"
dest_directory = "/tf/dataset/BACH"
sub_directory = ["Normal","Invasive","InSitu","Benign"]
for sub_dir in sub_directory:
    curr_sub_dir = os.path.join(src_directory,sub_dir)
    curr_filename = "data_file_" + sub_dir + ".csv"
    #curr_filename = "data_file_Fake_" + sub_dir + ".csv"
    content = []
    # extract image label
    for path,_,ls_files in os.walk(curr_sub_dir):
        for file in ls_files:
            if file.endswith('.tif'):
                label = extract_BACH_label(file)
                line = ("{},{}".format(os.path.join(curr_sub_dir,file),label)) 
                content.append(line)
    # write to file        
    with open(os.path.join(dest_directory,curr_filename), 'w') as writer:
        for line in content:
            writer.write(line + "\n")  

In [2]:
#create fake file csv from all fake images generated by pix2pix

#extract features
import numpy as np
def extract_fake_label(filename):
    parts = filename.split("_")
    if parts[1] == "B":
        label = 1
    else:
        label = 0
    sub_parts = parts[2].split("-")
    size = np.int(sub_parts[3])
    return label, size

import os
dest_path = "/tf/dataset/classes/"
content = []
output_filename = 'data_file_fake_pix2pix.csv'

#Write B cancer
src_path = "/tf/dataset/gen_breakhis"
folders = ['fake_40','fake_100','fake_200','fake_400']
for dir_path in folders:
    for path,_,files_list in os.walk(os.path.join(src_path,dir_path)):
        for filename in files_list:
            
            label, size = extract_fake_label(filename)
            full_filename = os.path.join(path,filename)
            line = ("{},{},{}".format(full_filename,label,size)) 
            content.append(line)

# write all to file        
with open(os.path.join(dest_path,output_filename), 'w') as writer:
    for line in content:
        writer.write(line + "\n")                